# Program Summary

This notebook generates the tables for the summary of eBPF programs.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, Tuple, List

import pandas as pd

from depsurf import DepKind, IssueEnum, Dep, DepReport
from utils import GRAY_DASH, load_pkl, save_latex, text_color, bold

COUNT = "Count"

KEYS = {
    COUNT: r"$\Sigma$",
    IssueEnum.ABSENT: r"$\varnothing$",
    IssueEnum.CHANGE: r"$\Delta$",
    IssueEnum.SELECTIVE_INLINE: "S",
    IssueEnum.FULL_INLINE: "F",
    IssueEnum.TRANSFORMATION: "T",
    IssueEnum.DUPLICATE: "D",
}

KINDS = {
    DepKind.FUNC: "Function",
    DepKind.STRUCT: "Struct",
    DepKind.FIELD: "Field",
    DepKind.TRACEPOINT: "Tracept",
    DepKind.SYSCALL: "Syscall",
}

COLUMNS = [
    (DepKind.FUNC, COUNT),
    (DepKind.FUNC, IssueEnum.ABSENT),
    (DepKind.FUNC, IssueEnum.CHANGE),
    (DepKind.FUNC, IssueEnum.FULL_INLINE),
    (DepKind.FUNC, IssueEnum.SELECTIVE_INLINE),
    (DepKind.FUNC, IssueEnum.TRANSFORMATION),
    (DepKind.FUNC, IssueEnum.DUPLICATE),
    (DepKind.STRUCT, COUNT),
    (DepKind.STRUCT, IssueEnum.ABSENT),
    (DepKind.FIELD, COUNT),
    (DepKind.FIELD, IssueEnum.ABSENT),
    (DepKind.FIELD, IssueEnum.CHANGE),
    (DepKind.TRACEPOINT, COUNT),
    (DepKind.TRACEPOINT, IssueEnum.ABSENT),
    (DepKind.TRACEPOINT, IssueEnum.CHANGE),
    (DepKind.SYSCALL, COUNT),
    (DepKind.SYSCALL, IssueEnum.ABSENT),
]

PROG_NAMES = {
    "tracee": "Tracee",
}

NUM_DEPS = r"\textbf{# uniq dep}"

data: Dict[str, Dict[Dep, DepReport]] = load_pkl("bcc")
data[NUM_DEPS] = {
    dep: report for prog_name, deps in data.items() for dep, report in deps.items()
}

table: Dict[str, Dict[Tuple[DepKind, str], int]] = {}

for prog_name, deps in data.items():
    if len(deps) == 0:
        print(f"Program {prog_name} has no dependencies")
        continue

    if prog_name in PROG_NAMES:
        prog_name = PROG_NAMES[prog_name]

    row = {col: 0 for col in COLUMNS}
    for dep, report in deps.items():
        kind = dep.kind
        if kind not in KINDS:
            continue
        row[(kind, COUNT)] += 1

        num_absent = sum(1 for issues in report.issue_dict.values() if IssueEnum.ABSENT in issues)
        row[(kind, IssueEnum.ABSENT)] += min(num_absent, 1)

        if kind not in [DepKind.STRUCT, DepKind.SYSCALL]:
            num_delta = sum(
                1 for issues in report.issue_dict.values() if IssueEnum.CHANGE in issues
            )
            row[(kind, IssueEnum.CHANGE)] += min(num_delta, 1)

        if kind == DepKind.FUNC:
            for issue in [
                IssueEnum.SELECTIVE_INLINE,
                IssueEnum.FULL_INLINE,
                IssueEnum.TRANSFORMATION,
                IssueEnum.DUPLICATE,
            ]:
                num_issue = sum(1 for issues in report.issue_dict.values() if issue in issues)
                row[(kind, issue)] += min(num_issue, 1)

    has_issue = False
    for (kind, key), count in row.items():
        if key == COUNT:
            continue
        if count != 0:
            has_issue = True
            continue

    if not has_issue:
        prog_name = text_color(prog_name, "blue")
    elif prog_name in ("biotop", "readahead"):
        prog_name = bold(prog_name)

    table[prog_name] = row

num_deps = table.pop(NUM_DEPS)
num_progs = {col: 0 for col in COLUMNS}
for prog_name, row in table.items():
    for (kind, key), count in row.items():
        if count != 0:
            num_progs[(kind, key)] += 1


df = pd.DataFrame(
    {
        tool: {(KINDS[kind], KEYS[key]): count for (kind, key), count in row.items()}
        for tool, row in table.items()
    }
).T

df = df.sort_values(by=df.columns.tolist(), ascending=False)
df = df.map(lambda x: GRAY_DASH if pd.isna(x) or x == 0 else int(x))
df.index.name = r"\multirow{2}{*}{Program}"
df = df.reset_index()

latex = df.to_latex(
    multicolumn_format="c|",
    column_format=r"l|rrrrrrr|rr|rrr|rrr|rr",
    index=False,
)

save_latex(latex, "bcc", midrule=False)
df

[ utils_pickle.py:18 ] INFO: Loding bcc from /Users/szhong/Code/DepSurf/output/bcc.pkl
[  utils_latex.py:118] INFO: Saved bcc to /Users/szhong/Code/DepSurf/paper/tabs/bcc.tex


Program bashreadline has no dependencies
Program gethostlatency has no dependencies
Program funclatency has no dependencies
Program ksnoop has no dependencies
Program javagc has no dependencies


\multirow{2}{*}{Program}              Function                                                                                                                                                    Struct                                       Field                                                           Tracept                                                           Syscall                      
                                             $\Sigma$         $\varnothing$              $\Delta$                     F                     S                     T                     D              $\Sigma$         $\varnothing$              $\Sigma$         $\varnothing$              $\Delta$              $\Sigma$         $\varnothing$              $\Delta$              $\Sigma$         $\varnothing$
0                        Tracee                    67                    14                    16                     5                    14                    14                     2                    98                    14                   250                    53                     9                    13                     3                     4                   446                   202
1                     klockstat                    14                     3  \color{lightgray}{-}  \color{lightgray}{-}                     4  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}
2                       vfsstat                     8  \color{lightgray}{-}                     5  \color{lightgray}{-}                     6                     1  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}
3               \textbf{biotop}                     5                     2                     2                     3                     2  \color{lightgray}{-}  \color{lightgray}{-}                     3  \color{lightgray}{-}                     7                     2                     1                     2                     2  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}
4                     cachestat                     5                     2                     2  \color{lightgray}{-}                     1  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}                     2                     2                     1  \color{lightgray}{-}  \color{lightgray}{-}
5                        fsdist                     5                     2                     1  \color{lightgray}{-}                     2                     2  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}
6                     tcptracer                     5  \color{lightgray}{-}                     1  \color{lightgray}{-}  \color{lightgray}{-}                     3  \color{lightgray}{-}                     6  \color{lightgray}{-}                    14  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}
7            \textbf{readahead}                     4                     3                     1                     2                     3                     1                     1                     2                     1                     1                     1  \color{lightgray}{-}  \color{lightgray}{-}  \color{lightgray}{-}  \color{light

In [3]:
from utils import mini_bar, multirow

KINDS = {
    **KINDS,
    DepKind.FUNC: "Func",
}

MULTIROW = (DepKind.FIELD, IssueEnum.ABSENT)

ISSUES = {
    (DepKind.FUNC, IssueEnum.ABSENT): [3687, 3695, 3692, 3747, 4337, 4885],
    (DepKind.FUNC, IssueEnum.CHANGE): [1911, 3360, 4339, 4340, 4346],
    (DepKind.FUNC, IssueEnum.SELECTIVE_INLINE): [703, 1667, 2252, 2373, 3913],
    (DepKind.FUNC, IssueEnum.FULL_INLINE): [4261, 4478, 4638, 4700, 5115],
    (DepKind.FUNC, IssueEnum.TRANSFORMATION): [1754, 3293, 3339, 3315, 4937],
    (DepKind.STRUCT, IssueEnum.ABSENT): [4340, 4339],
    (DepKind.FIELD, IssueEnum.ABSENT): [
        1384,
        3612,
        3647,
        3650,
        3658,
        3672,
        3680,
        3859,
        3903,
        3926,
        3936,
    ],
    (DepKind.FIELD, IssueEnum.CHANGE): [3845, 3865],
    (DepKind.TRACEPOINT, IssueEnum.ABSENT): [1636, 2816, 4384],
    (DepKind.TRACEPOINT, IssueEnum.CHANGE): [3317, 3338, 4076, 4476],
    (DepKind.SYSCALL, IssueEnum.ABSENT): [3012, 3668, 3843, 4287],
}


PROG = "# Programs"
UNIQ_DEP = "# Uniq Deps"
BUG = "Reported Issue No."

COLORS = {
    PROG: "prog",
    UNIQ_DEP: "dep",
}

table = {}
for name, data in [(PROG, num_progs), (UNIQ_DEP, num_deps)]:
    row = {}
    for (kind, key), count in data.items():
        text = mini_bar(
            text=count,
            percent=count / data[(kind, COUNT)],
            total_width=1.5,
            color=COLORS[name],
            bg_color="bg",
        )
        if (kind, key) == MULTIROW:
            row[(KINDS[kind], multirow(KEYS[key]))] = multirow(text)
            row[(KINDS[kind], "")] = ""
        else:
            row[(KINDS[kind], KEYS[key])] = text
    table[name] = row


def bccissues(issues: List[int]) -> str:
    return ", ".join(f"\\bccissue{{{issue}}}" for issue in issues)


row = {}
for (kind, key), issues in ISSUES.items():
    if (kind, key) == MULTIROW:
        n = (len(issues) + 1) // 2
        row[(KINDS[kind], multirow(KEYS[key]))] = bccissues(issues[:n])
        row[(KINDS[kind], "")] = bccissues(issues[n:])
    else:
        row[(KINDS[kind], KEYS[key])] = bccissues(issues)
table[BUG] = row


df = pd.DataFrame(table)
df = df.fillna(GRAY_DASH)
latex = df.to_latex(column_format="llrrr")
save_latex(latex, "summary", rotate=False)
df

[  utils_latex.py:118] INFO: Saved summary to /Users/szhong/Code/DepSurf/paper/tabs/summary.tex


# Programs                                                                                                      # Uniq Deps                                                                                    Reported Issue No.
Func    $\Sigma$                                               \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{1.50cm}][r]{25}}                      \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{1.50cm}][r]{126}}                                                                                  \color{lightgray}{-}
        $\varnothing$                                          \colorbox{bg}{\makebox[{0.90cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.60cm}][r]{10}}                       \colorbox{bg}{\makebox[{1.15cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.35cm}][r]{29}}  \bccissue{3687}, \bccissue{3695}, \bccissue{3692}, \bccissue{3747}, \bccissue{4337}, \bccissue{4885}
        $\Delta$                                               \colorbox{bg}{\makebox[{0.66cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.84cm}][r]{14}}                       \colorbox{bg}{\makebox[{1.13cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.37cm}][r]{31}}                   \bccissue{1911}, \bccissue{3360}, \bccissue{4339}, \bccissue{4340}, \bccissue{4346}
        F                                                       \colorbox{bg}{\makebox[{1.14cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.36cm}][r]{6}}                       \colorbox{bg}{\makebox[{1.37cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.13cm}][r]{11}}                   \bccissue{4261}, \bccissue{4478}, \bccissue{4638}, \bccissue{4700}, \bccissue{5115}
        S                                                      \colorbox{bg}{\makebox[{0.66cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.84cm}][r]{14}}                       \colorbox{bg}{\makebox[{1.12cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.38cm}][r]{32}}                    \bccissue{703}, \bccissue{1667}, \bccissue{2252}, \bccissue{2373}, \bccissue{3913}
        T                                                      \colorbox{bg}{\makebox[{0.66cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.84cm}][r]{14}}                       \colorbox{bg}{\makebox[{1.17cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.33cm}][r]{28}}                   \bccissue{1754}, \bccissue{3293}, \bccissue{3339}, \bccissue{3315}, \bccissue{4937}
        D                                                       \colorbox{bg}{\makebox[{1.38cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.12cm}][r]{2}}                        \colorbox{bg}{\makebox[{1.46cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.04cm}][r]{3}}                                                                                  \color{lightgray}{-}
Struct  $\Sigma$                                               \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{1.50cm}][r]{43}}                      \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{1.50cm}][r]{135}}                                                                                  \color{lightgray}{-}
        $\varnothing$                                          \colorbox{bg}{\makebox[{1.05cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{0.45cm}][r]{13}}                       \colorbox{bg}{\makebox[{1.16cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{0.34cm}][r]{31}}                                                                      \bccissue{4340}, \bccissue{4339}
Field   $\Sigma$                                               \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{prog}{\makebox[{1.50cm}][r]{43}}                      \colorbox{bg}{\makebox[{0.00cm}][r]{\phantom{0}}}\colorbox{dep}{\makebox[{1.50cm}][r]{342}}                                                                                  \color{lightgray}{-}
        \multirow[c]{2}{*}{$\varnothing$}  \multirow[c]{2}{*}{\colorbox{bg